# 課題1
> 既存のルーティングプロトコルとOpenFlowの違いについて自分なりにまとめてください。

既存のルーティングプロトコルは分散型で各ルーターが自律的に動作するインターネットのようなものでスケーラビリティがいいが柔軟なルーティングや制御は難しい一方、OpenFlowはスイッチというノードをコントローラーが一元管理する中央集権型でソフトウェアから柔軟に制御できる

# 課題2
>OSPFの問題: 以下のネットワークでルーターBの経路表はどのようになるか解答してください。

```mermaid
graph LR
    A ---|2| B ---|3| E
    A ---|1| C
    B ---|1| D
    C ---|3| D ---|1| E
```

ただし、正解データは以下の形式で記述してください。(以下はルーターAの経路表)
```
{destination network=B: next hop=B}
{destination network=C: next hop=C}
{destination network=D: next hop=B}
{destination network=E: next hop=B}
```

# 課題3
OpenFlowでのフローに関する問題
>- フローを自分で定義し、本キャプチャデータにおける各フローのパケット数を調べて回答してください。
>  - フローの例を記載
>    1. (srcIP, dstIP)で定義
>      - srcIP=192.168.1.2, dstIP=192.168.3.5
>      - srcIP=192.168.1.7, dstIP=216.58.196.228
>      - ...
>    2. (srcIP,  srcPort, dstIP, dstPort, protocol)で定義
>      - srcIP=192.168.1.2, srcPort=60006, dstIP=192.168.3.5, dstPort=80, protocol=TCP
>      - srcIP=192.168.1.2, srcPort=50000, dstIP=192.168.3.12, dstPort=443, protocol=TCP
>      - srcIP=192.168.1.7, srcPort=50011, dstIP=216.58.196.228, dstPort=80, protocol=TCP
>      - ...
>  - IPv6は無視してOK
>- またその調査結果を踏まえて、(宛先IPアドレス毎にルーティングした場合と比べて)フロー単位で制御するときめ細かな制御ができる理由を自分なりに考えてのべてください。
>- ヒント: WireSharkの「統計」>「対話」を選ぶといろんなレベルでパケット数が確認できます。

了解です！この課題は **OSPF（Shortest Path First）** の最短経路計算を、与えられたリンクコスト（辺の数字）で行い、**ルーターBから各宛先ネットワークへの「最初の一歩＝ネクストホップ」**を答える問題です。

* 図の数字は **リンクコスト**（小さいほど好ましい）
* OSPFは各ルーターが自分を始点に **最短コスト経路** を計算（ダイクストラ）
* 経路表に載せる「next hop」は、**最短経路の“最初に出る隣接ルーター”**

### 計算（Bを始点）

* **A**: 直結 B–A=2 が最短（他経路は 5 以上）→ next hop **A**
* **C**: B→A→C が 2+1=**3**、B→D→C は 1+3=4 → next hop **A**
* **D**: 直結 B–D=**1** が最短 → next hop **D**
* **E**: B→D→E が 1+1=**2**、直結 B–E=3 より安い → next hop **D**

### 解答（ルーターBの経路表）

```
{destination network=A: next hop=A}
{destination network=C: next hop=A}
{destination network=D: next hop=D}
{destination network=E: next hop=D}
```

ポイント：
E行きが「直結のEではなくD経由」になるのは、**合計コスト 2（B→D→E）が 3（B→E）より小さい**からです。これこそがOSPFの「トポロジ全体の最短経路」を選ぶ動きです。


In [17]:
import polars as pl
pl.Config().set_tbl_rows(50)

df_cap = pl.read_csv("./tmp.csv")
df_cap.head()

アドレス A,アドレス B,パケット,バイト,ストリームID,Packets A → B,Bytes A → B,Packets B → A,Bytes B → A,相対的な開始,時間,Bits/s A → B,Bits/s B → A
str,str,i64,i64,i64,i64,i64,i64,i64,f64,f64,str,str
"""74.125.23.189""","""192.168.3.8""",418,63237,2,265,34905,153,28332,0.19995,103.27997,"""2703""","""2194"""
"""162.125.34.129""","""192.168.3.8""",20,7454,23,12,1676,8,5778,25.565319,78.370895,"""171""","""589"""
"""192.168.3.2""","""192.168.3.8""",18,6138,1,18,6138,0,0,0.139771,5.118914,"""9592""","""0"""
"""192.168.3.2""","""224.0.0.251""",1,381,22,1,381,0,0,21.714496,0.0,"""""",""""""
"""192.168.3.2""","""239.255.255.250""",6,2124,28,6,2124,0,0,56.120639,0.306395,"""55457""","""0"""


In [18]:
df_cap.select("アドレス A", "アドレス B", "パケット")

アドレス A,アドレス B,パケット
str,str,i64
"""74.125.23.189""","""192.168.3.8""",418
"""162.125.34.129""","""192.168.3.8""",20
"""192.168.3.2""","""192.168.3.8""",18
"""192.168.3.2""","""224.0.0.251""",1
"""192.168.3.2""","""239.255.255.250""",6
"""192.168.3.4""","""224.0.0.251""",1
"""192.168.3.4""","""239.255.255.250""",12
"""192.168.3.7""","""224.0.0.251""",2
"""192.168.3.8""","""23.10.7.86""",97


本キャプチャデータでは、同一ホスト（192.168.3.8）から多様な宛先への通信が観測され、パケット数や通信方向に大きな差が見られた。
宛先IPアドレス単位でのルーティングでは、これらすべての通信が同一の経路・制御ポリシーで扱われるため、通信特性に応じた最適な制御が困難である。
一方、OpenFlowのようにフロー（srcIP, srcPort, dstIP, dstPort, protocol）単位で制御すれば、アプリケーション種別や通信方向ごとに異なる経路選択・帯域制御・優先度設定が可能となり、きめ細かなトラフィック管理が実現できる。